In [46]:
#for testing purposes
#%run BaseRenderer.ipynb

def get_bms_token(bms_user, bms_password, DEBUG=DEBUG):
    import requests
    import json

    url = "https://brapi.bms-uat-test.net/bmsapi/brapi/v1/token"

    payload = json.dumps({
      "username": f"{bms_user}",
      "password": f"{bms_password}",
      "grant_type": "",
      "client_id": ""
    })
    headers = {
      'Content-Type': 'application/json'
    }
    if DEBUG:
        print("DEBUG MODE",bms_user,  bms_password)
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)
    access_token = json.loads(response.text)['access_token']
    return access_token


app_title = v.CardTitle(class_='indigo white--text text-h5',
                        children=['Data Sources Selector'])

addToRenderer(app_title)

class AltertTitle(v.VuetifyTemplate):
    template = Unicode('''
             <v-alert dense
                  outlined
                  type="error"
             >
              Please select provider
            </v-alert>''').tag(sync=True)
  
    def vue_menu_click(self, data):
        pass
  
alert = AltertTitle()

provider_alert = v.Alert(dismissible=True, type="error", color='pink darken-1', dark=True, border="bottom", children=["Data provider not selected."], v_model=False)

class ProviderDialog:
    def __init__(self, provider_name=None, external_observer=None, provider_alert=provider_alert):
        self.d_user = v.TextField(label='Username', placeholder='username', v_model="")
        self.d_pass = v.TextField(label='Password', placeholder='password', v_model="", type='password')
        self.submit_btn = v.Btn(color='primary', children=['Submit'])
        self.login_btn = v.Btn(color='success', children=['Login'])
        self.provider_name = provider_name
        self.provider_alert = provider_alert
        if provider_name:
            title = f'{provider_name} Login'
        else:
            title = 'Login'
        self.provider_title = v.CardTitle(class_='headline gray lighten-2', primary_title=True, children=[title])
        
        self.dialog = v.Dialog(width='500',
            #v_model='dialog',
            children=[
                v.Card(children=[
                    self.provider_title,
                    v.CardText(children=[
                        "Please enter your credentials below",
                        self.d_user, self.d_pass, 
                        self.submit_btn
                    ])
            ])
        ])
        
        self.submit_btn.on_event('click', self.on_submit_btn_clicked)
        self.login_btn.on_event('click', self.on_login_btn_clicked)
        if external_observer:
            external_observer.on_event('click', self.on_login_btn_clicked)
            self.external_observer = external_observer
            
            
        self.dialog.v_model = False
        
        self.layout = v.Layout(children=[
            self.dialog, self.login_btn])
    def get_layout(self):
        self.layout = v.Container(children=[
            self.dialog])
        return self.layout
    
    def get_layout_standalone(self):
        self.layout = v.Layout(children=[
            self.dialog, self.login_btn])
        return self.layout
    
    def set_provider(self, provider_name, update_title=True):
        self.provider_name = provider_name
        if update_title:
            self.set_dialog_title(f'{provider_name} Login')
            
    
    def set_dialog_title(self, title):
        self.provider_title.children = self.provider_title.children[0:0] + [title]
        print("UPDATING TITLE", self.provider_title.children)



    def on_submit_btn_clicked(self, *args):
        #O = HTML('''<script>toggle_show_password();</script>''')
        self.dialog.v_model = False
        #display(O)
        print("DATA", args)
        try:
            self.d_token = get_bms_token(self.d_user.v_model, self.d_pass.v_model)
            print("TOKEN", self.d_token)
            if self.d_token:
                self.external_observer.color = "success"
            else:
                self.external_observer.color = "error"
        except:
             self.external_observer.color = "error"



    def on_login_btn_clicked(self, *args):
        if not self.provider_name:
            print("Provider not selected")
            self.provider_alert.v_model = True
            return
        self.dialog.v_model = True
        

    
provider_pill_geno = v.Btn(color='active', rounded=True, class_='ma-2', children=['geno:N/A'])
provider_pill_pheno = v.Btn(color='active', rounded=True, class_='ma-2', children=['pheno:N/A'])

containter = v.Container(children=[
    #genotype_provider_select,
    #genotype_provider_display,
    #active_providers['geno'].layout,
    provider_pill_geno,
    provider_pill_pheno
    
])


info = v.Chip(class_='ma-2')
#<span class="text-h5 white--text">Sarah Mcbeal</span>
layout = v.Layout(children=[
#    v.Card()
     v.Html(
        tag='span',
        #attributes={'title': 'a title'},
        _class="text-h5 white--text",
        children=['Selected Providers']
    ),
    containter, 
])

addToRenderer(layout)


active_providers = {'geno':None, 'pheno':None, 'metadata':None}
genotype_provider_select = v.Select(
    v_model='',
    label='Genotype Data Provider',
    items=['GOBII', 'GIGWA', 'Matrix:GT', 'MontyDB'])

phenotype_provider_select = v.Select(
    v_model='',
    label='Phenotype Data Provider',
    items=['BMS', 'BreedBase', 'Custom'])

active_providers['geno'] = ProviderDialog(external_observer=provider_pill_geno)
active_providers['pheno'] = ProviderDialog(external_observer=provider_pill_pheno)

# genotype_provider_display = v.Chip()
#jslink((genotype_provider_select, 'v_model'), (genotype_provider_display, 'color'))

#selection = ["N/A"]
#jslink((genotype_provider_select, 'v_model'), (provider_pill, selection))

def on_update_provider_geno(widget, event, data):
    print(widget, event, data)
    provider_pill_geno.children = [f"geno:{data}"]
    active_providers['geno'].set_provider(data, update_title=True)

    
def on_update_provider_pheno(widget, event, data):
    print(widget, event, data)
    provider_pill_pheno.children = [f"pheno:{data}"]
    active_providers['pheno'].set_provider(data, update_title=True)

    
genotype_provider_select.on_event('change', on_update_provider_geno)
phenotype_provider_select.on_event('change', on_update_provider_pheno)


selector_container = v.Container(children=[
    genotype_provider_select,
    phenotype_provider_select,

])


layout = v.Layout(children=[
     v.Html(
        tag='p',
        attributes={'title': 'a title'},
        children=['Choose Provider']
    ),
    selector_container,
    active_providers['geno'].dialog,active_providers['pheno'].dialog,

])
addToRenderer(layout)

provider_alert

# createss diaglog and submit button
# it also hooks the diaglog to the function that will get the BrAPI token
import ipyvuetify as v

d_user = v.TextField(label='Username', placeholder='username', v_model="")
d_pass = v.TextField(label='Password', placeholder='password', v_model="", type='password')

submit_btn = v.Btn(color='primary', children=['Submit'])


def on_submit_btn_clicked(*args):
    dialog.v_model = False
    d_token = get_bms_token(d_user.v_model, d_pass.v_model)
    print("Obtained token", d_token)
    
    
submit_btn.on_event('click', on_submit_btn_clicked)

new_btn = v.Btn(color='success', children=['Login'])
def on_new_btn_clicked(*args):
    dialog.v_model = True
    
new_btn.on_event('click', on_new_btn_clicked)
dialog = v.Dialog(width='500',
    #v_model='dialog',
    children=[
        v.Card(children=[
            v.CardTitle(class_='headline gray lighten-2', primary_title=True, children=[
                "DBMS Login"
            ]),
            v.CardText(children=[
                "Please enter your credentials below",
                d_user, d_pass, 
                #d_save,
                submit_btn
            ])
    ])
])
dialog.v_model = False

dialog.on_event('keydown.stop', lambda *args: None)

# 

d_user = v.TextField(label='Username', placeholder='user name', v_model="")
d_pass = v.TextField(label='Password', placeholder='password', v_model="")#, type='password')
d_save = v.Btn(color='success', dark=True, children=['Save'], v_model=False )
d_token = ''
    
login_form = v.Dialog(width='500', user_name=None, user_pass=None, v_model=False,
    v_slots=[{
        'name': 'activator',
        'variable': 'loginform',
        'children': v.Btn(
            v_on='loginform.on', 
            color='success', dark=True, children=[
            'Login'
        ]),
    }], 
    children=[
        v.Card(children=[
            v.CardTitle(class_='headline gray lighten-2', primary_title=True, 
                        children=["DBMS Login"
            ]),
            v.CardText(children=[
                "Please enter your credentials below",
                d_user, d_pass, d_save

            ]),
    ]),
])

def update_user(widget, event, data):
    login_form.user_name = data

def on_save(widget, event, data, dialog): 
    # print(d_user.v_model, d_pass.v_model, login_form.v_model)
    dialog.v_model = False
    login_form.v_model = False
   
    
def on_model(*args):
    # print(args)
    d_token = get_bms_token(d_user.v_model, d_pass)
    # print("TOKEN", d_token)
    login_form.v_model = False

login_form.on_event('update:userName', update_user)

#for testing purposes
#return v.Container(children=_RENDERER)

SyntaxError: 'return' outside function (2543997246.py, line 310)

Container(children=[CardTitle(children=['Data Sources Selector'], class_='indigo white--text text-h5'), Layout…